In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("gsk.......PTO")

In [2]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('corona.txt')
documents = loader.load()

In [3]:
documents

[Document(metadata={'source': 'corona.txt'}, page_content='Coronavirus, also known as COVID-19, is a highly contagious virus that emerged in late 2019. It primarily spreads through respiratory droplets when an infected person coughs, sneezes, or talks. Common symptoms include fever, cough, fatigue, and difficulty breathing, though some people may be asymptomatic.\n\nPreventive measures are essential to reduce transmission, such as wearing masks, maintaining social distancing, practicing hand hygiene, and getting vaccinated. COVID-19 can lead to severe illness in older adults and people with underlying health conditions.\n\nGovernments worldwide have implemented strategies like lockdowns, travel restrictions, and mass vaccination campaigns to control the spread of the virus. Ongoing research continues to improve treatments and vaccines to combat the pandemic effectively.\n')]

In [4]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [5]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='Coronavirus, also known as COVID-19, is a highly contagious virus that emerged in late 2019. It
primarily spreads through respiratory droplets when an infected person coughs, sneezes, or talks. Common
symptoms include fever, cough, fatigue, and difficulty breathing, though some people may be asymptomatic.

Preventive measures are essential to reduce transmission, such as wearing masks, maintaining social
distancing, practicing hand hygiene, and getting vaccinated. COVID-19 can lead to severe illness in older
adults and people with underlying health conditions.

Governments worldwide have implemented strategies like lockdowns, travel restrictions, and mass vaccination
campaigns to control the spread of the virus. Ongoing research continues to improve treatments and vaccines to
combat the pandemic effectively.
' metadata={'source': 'corona.txt'}


In [6]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [7]:
len(docs)

1

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

c:\Users\abhin\FSDS\RAG\rag_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\abhin\FSDS\RAG\rag_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [10]:
query = "explain about indian premier league"
docs = db.similarity_search(query)

In [11]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Coronavirus, also known as COVID-19, is a highly contagious virus that emerged in late 2019. It primarily
spreads through respiratory droplets when an infected person coughs, sneezes, or talks. Common symptoms
include fever, cough, fatigue, and difficulty breathing, though some people may be asymptomatic.

Preventive measures are essential to reduce transmission, such as wearing masks, maintaining social
distancing, practicing hand hygiene, and getting vaccinated. COVID-19 can lead to severe illness in older
adults and people with underlying health conditions.

Governments worldwide have implemented strategies like lockdowns, travel restrictions, and mass vaccination
campaigns to control the spread of the virus. Ongoing research continues to improve treatments and vaccines to
combat the pandemic effectively.


In [12]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
    groq_api_key="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",  # pass your actual key
    model="gemma2-9b-it"
)

In [13]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(GROQ_LLM, chain_type="stuff")

In [14]:
query = "tell me about corona virus"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

c:\Users\abhin\FSDS\RAG\rag_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Coronavirus, also known as COVID-19, is a highly contagious virus that emerged in late 2019. It spreads through respiratory droplets when an infected person coughs, sneezes, or talks.  \n\nCommon symptoms include fever, cough, fatigue, and difficulty breathing, but some people may not show any symptoms. \n\nPreventing its spread involves wearing masks, social distancing, good hand hygiene, and getting vaccinated. \n\nCOVID-19 can cause severe illness in older adults and people with health problems. \n\nGovernments worldwide have used strategies like lockdowns, travel restrictions, and mass vaccination to try and control the virus.  Research is ongoing to develop better treatments and vaccines. \n'

In [15]:
query = "Tell me about covaccine"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'I don\'t have information about a specific vaccine called "covaccine." \n\nIt\'s possible this is a regional or lesser-known vaccine name. To get accurate information, I recommend checking with reliable health sources like the World Health Organization (WHO) or your local health department. \n'